<a href="https://colab.research.google.com/github/amantino69/Tickets/blob/main/C%C3%B3pia_de_Untitled26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files

# Fazer upload do arquivo JSON
uploaded = files.upload()


In [ ]:
from google.cloud import storage
import pandas as pd
from io import StringIO

# Função para carregar o CSV do Google Cloud Storage
def load_csv_from_gcs(bucket_name, file_path):
    # Explicitly create a client with the service account credentials
    client = storage.Client.from_service_account_json("/content/drive/MyDrive/AI/egon-425511-86aad078a288.json")  # Replace with the actual path
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(file_path)
    data = blob.download_as_text()
    df = pd.read_csv(StringIO(data))
    return df

# Configuração do GCS
bucket_name = 'bucket_egon'
# Update file_path to point to the file *within* your GCS bucket
file_path = 'whatsapp_qa_dataset.csv'  # Example: assuming the file is at the root of your bucket

# Carregar o CSV
df = load_csv_from_gcs(bucket_name, file_path)

# Exibir as primeiras linhas do DataFrame
print(df.head())

                                            Question  \
0  Já paguei. Você está na escola? Tomou a injeçã...   
1  Outra coisa... sobre 3 vendas boas entende -se...   
2                         Nossa, que convite é esse?   
3                           pq vc mandou 2 arquivos?   
4              pq trocou de nome?ta sendo procurado?   

                                              Answer  
0  Duas horas por dia pra lua n da nem pra mandar...  
1  Oi. A Ju disse que tem um monte de restaurante...  
2  Vou recusar, pq mais q nunca sou uma mulher mt...  
3         Esqueci de te falar da norma pra impressao  
4                                                 bj  


In [ ]:
import os

# Substitua 'your_service_account.json' pelo nome do arquivo JSON carregado
service_account_key = '/content/drive/MyDrive/AI/egon-425511-86aad078a288.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_key

In [ ]:
# Carregar os dados do CSV
csv_path = '/content/drive/MyDrive/AI/whatsapp_qa_dataset.csv'
df = pd.read_csv(csv_path)

# Exibir as primeiras linhas do DataFrame para verificar os dados
print(df.head())


                                            Question  \
0  Já paguei. Você está na escola? Tomou a injeçã...   
1  Outra coisa... sobre 3 vendas boas entende -se...   
2                         Nossa, que convite é esse?   
3                           pq vc mandou 2 arquivos?   
4              pq trocou de nome?ta sendo procurado?   

                                              Answer  
0  Duas horas por dia pra lua n da nem pra mandar...  
1  Oi. A Ju disse que tem um monte de restaurante...  
2  Vou recusar, pq mais q nunca sou uma mulher mt...  
3         Esqueci de te falar da norma pra impressao  
4                                                 bj  


In [ ]:
import re

def limpar_texto(texto):
  texto = texto.lower()
  texto = re.sub(r'[^\w\s]', '', texto)  # Remove pontuação
  return texto

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Nome da coluna que contém os textos (substitua pelo nome correto após verificar)
coluna_texto = 'Question'

# Adicionar tokens especiais aos textos
df['texto_with_tokens'] = df[coluna_texto].apply(lambda x: 'starttoken ' + str(x) + ' endtoken')

# Verificar as primeiras linhas para confirmar que os tokens foram adicionados
print(df[[coluna_texto, 'texto_with_tokens']].head())

# Definir o tokenizer
tokenizer = Tokenizer()

# Ajustar o tokenizer nos textos com tokens especiais
tokenizer.fit_on_texts(df['texto_with_tokens'])

# Verificar se os tokens especiais foram adicionados
print('starttoken' in tokenizer.word_index)
print('endtoken' in tokenizer.word_index)

# Exibir o vocabulário do tokenizer
print(tokenizer.word_index)


                                            Question  \
0  Já paguei. Você está na escola? Tomou a injeçã...   
1  Outra coisa... sobre 3 vendas boas entende -se...   
2                         Nossa, que convite é esse?   
3                           pq vc mandou 2 arquivos?   
4              pq trocou de nome?ta sendo procurado?   

                                   texto_with_tokens  
0  starttoken Já paguei. Você está na escola? Tom...  
1  starttoken Outra coisa... sobre 3 vendas boas ...  
2     starttoken Nossa, que convite é esse? endtoken  
3       starttoken pq vc mandou 2 arquivos? endtoken  
4  starttoken pq trocou de nome?ta sendo procurad...  
True
True
{'starttoken': 1, 'endtoken': 2, 'que': 3, 'o': 4, 'a': 5, 'e': 6, 'de': 7, 'vc': 8, 'não': 9, 'pra': 10, 'é': 11, 'q': 12, 'para': 13, 'da': 14, 'com': 15, 'tem': 16, 'ou': 17, 'do': 18, 'vai': 19, 'no': 20, 'na': 21, 'você': 22, 'se': 23, 'ele': 24, 'por': 25, 'como': 26, 'ta': 27, 'n': 28, 'em': 29, 'mas': 30, 'um': 31

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Attention
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.cloud import storage
import uuid
import numpy as np

# Função para carregar o CSV do Google Cloud Storage
def load_csv_from_gcs(bucket_name, file_path):
    client = storage.Client()
    bucket = client.bucket(bucket_name)

    # Verifique se o nome do arquivo está correto e se ele existe no bucket.
    # Ajuste o 'file_path' se necessário.
    blob = bucket.blob('whatsapp_qa_dataset.csv')  # Removido 'content/' do caminho

    data = blob.download_as_text()
    df = pd.read_csv(StringIO(data))
    return df

# Configuração do GCS
bucket_name = 'bucket_egon'  # Verifique se o nome do bucket está correto
file_path = 'whatsapp_qa_dataset.csv'  # Ajuste o caminho do arquivo se necessário

# Carregar o CSV
data = load_csv_from_gcs(bucket_name, file_path)


# Configuração do GCS
bucket_name = 'bucket_egon'
file_path = 'content/whatsapp_qa_dataset.csv'

# Carregar o CSV
data = load_csv_from_gcs(bucket_name, file_path)

# Pré-processamento dos dados
data['Question'] = data['Question'].astype(str)
data['Answer'] = 'starttoken ' + data['Answer'].astype(str) + ' endtoken'

# Dividir os dados em conjuntos de treinamento e validação
train_data, val_data = train_test_split(data, test_size=0.2)

# Tokenização
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(train_data['Question'].tolist() + train_data['Answer'].tolist())

# Converter textos em sequências
train_questions = tokenizer.texts_to_sequences(train_data['Question'].tolist())
train_answers = tokenizer.texts_to_sequences(train_data['Answer'].tolist())
val_questions = tokenizer.texts_to_sequences(val_data['Question'].tolist())
val_answers = tokenizer.texts_to_sequences(val_data['Answer'].tolist())

# Verificar se as sequências foram geradas corretamente
print("Exemplo de sequência de perguntas de treino:", train_questions[0])
print("Exemplo de sequência de respostas de treino:", train_answers[0])

# Verificar o tipo e conteúdo das sequências
print("Tipo de train_questions:", type(train_questions))
print("Conteúdo de train_questions:", train_questions[:5])  # Imprime os primeiros 5 elementos


# Padding das sequências
max_len = max(max(len(seq) for seq in train_questions if isinstance(seq, list)),
              max(len(seq) for seq in train_answers if isinstance(seq, list)),
              max(len(seq) for seq in val_questions if isinstance(seq, list)),
              max(len(seq) for seq in val_answers if isinstance(seq, list)))




# Pad the sequences to the same length
train_questions = pad_sequences(train_questions, maxlen=max_len, padding='post')
train_answers = pad_sequences(train_answers, maxlen=max_len, padding='post')
val_questions = pad_sequences(val_questions, maxlen=max_len, padding='post')
val_answers = pad_sequences(val_answers, maxlen=max_len, padding='post')


# Variável de tamanho do vocabulário
vocab_size = len(tokenizer.word_index) + 1

# Definir o modelo
embedding_dim = 256
units = 512

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(units, return_state=True, return_sequences=True)(encoder_embedding)
encoder_states = [state_h, state_c]


# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(decoder_inputs)

# Use the encoder output as the initial state for the decoder
# Pass the encoder LSTM's output sequence as input to the decoder LSTM
decoder_lstm, _, _ = LSTM(units, return_sequences=True, return_state=True)(
    decoder_embedding, initial_state=encoder_states
)

attention = Attention()([encoder_lstm, decoder_lstm])

# Reshape the attention output to match the decoder LSTM output
attention_reshaped = tf.keras.layers.Reshape((-1, units))(attention)

# ... (rest of your code)

# Concatenate the attention output with the decoder LSTM output
# ... (rest of your code)

# Concatenate the attention output with the decoder LSTM output
# Ensure both tensors have the same shape on the second dimension
decoder_concat_input = tf.concat([decoder_lstm, attention_reshaped[:, :-1, :]], axis=-1)  # Adjust attention_reshaped

# Add a Dense layer to produce the final output probabilities
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat_input)  # Define decoder_outputs

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# ... (rest of your code)
# ... (rest of your code)
# Compilar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Preparar os dados alvo para o treinamento
target_data = train_answers[:, 1:]
train_answers = train_answers[:, :-1]

val_target_data = val_answers[:, 1:]
val_answers = val_answers[:, :-1]

# Adicionar uma dimensão extra aos dados alvo
target_data = target_data[..., None]
val_target_data = val_target_data[..., None]

# Treinar o modelo
batch_size = 64
epochs = 52

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.fit([train_questions, train_answers], target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=([val_questions, val_answers], val_target_data),
          callbacks=[EarlyStopping(monitor='val_loss', patience=3),
                     tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2),
                     tf.keras.callbacks.ModelCheckpoint('melhor_modelo.h5', save_best_only=True)])

# Salvar o modelo localmente
model.save('whatsapp_response_model.h5')

# Função para fazer upload do modelo para o Google Cloud Storage
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Faz upload de um arquivo para o bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"Arquivo {source_file_name} enviado para {destination_blob_name}.")

# Definir o nome do bucket e o caminho de destino
bucket_name = 'nome-do-seu-bucket'
source_file_name = 'whatsapp_response_model.h5'
destination_blob_name = 'models/whatsapp_response_model.h5'

# Fazer upload do modelo
upload_blob(bucket_name, source_file_name, destination_blob_name)

# Carregar o modelo
model = tf.keras.models.load_model(source_file_name)

# Definir o modelo do encoder para inferência
encoder_model = Model(encoder_inputs, encoder_states)

# Definir o modelo do decoder para inferência
decoder_state_input_h = Input(shape=(units,))
decoder_state_input_c = Input(shape=(units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def decode_sequence(input_seq, temperatura=0.7):
    # Encode the input as state vectors
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1
    target_seq = np.zeros((1, 1))

    # Populate the first character of target sequence with the start token
    target_seq[0, 0] = tokenizer.word_index['starttoken']

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Apply temperature sampling
        token_probs = output_tokens[0, -1, :] / temperatura
        token_probs = np.exp(token_probs) / np.sum(np.exp(token_probs))

        # Sample a token
        sampled_token_index = np.random.choice(range(vocab_size), p=token_probs)
        if sampled_token_index == 0:
            print("Model predicted a token index of 0. This may indicate an issue with the model or the input data.")
            break
        sampled_char = tokenizer.index_word[sampled_token_index]
        decoded_sentence += ' ' + sampled_char

        # Exit condition: either hit max length or find stop token
        if (sampled_char == 'endtoken' or len(decoded_sentence) > max_len):
            stop_condition = True

        # Update the target sequence (length 1)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

# Test the model
test_input = "Sua pergunta aqui"
test_input_seq = tokenizer.texts_to_sequences([test_input])
test_input_seq = tf.keras.preprocessing.sequence.pad_sequences(test_input_seq, maxlen=max_len, padding='post')

response = decode_sequence(test_input_seq)
print(response)


Exemplo de sequência de perguntas de treino: [49, 416, 6, 417, 14, 831, 169, 651, 19, 832, 5, 32, 106, 6, 417, 14, 1161]
Exemplo de sequência de respostas de treino: [1, 6, 1947, 73, 10, 5461, 5, 1217, 2642, 57, 247, 2]
Tipo de train_questions: <class 'list'>
Conteúdo de train_questions: [[49, 416, 6, 417, 14, 831, 169, 651, 19, 832, 5, 32, 106, 6, 417, 14, 1161], [80, 7, 3694, 13, 14, 397, 23, 22, 27, 177, 1536], [214, 39, 320, 466, 2436, 20, 294, 29, 23, 4, 156, 44], [70, 47, 1315, 1871, 833, 6, 761], [762, 14, 25, 2437, 45, 5, 2437, 3695, 2438, 221, 1162, 15, 1872, 3696]]
Epoch 1/52
52/52 [==============================] - ETA: 0s - loss: 0.9585 - accuracy: 0.9606

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


52/52 [==============================] - 31s 530ms/step - loss: 0.9585 - accuracy: 0.9606 - val_loss: 0.1525 - val_accuracy: 0.9808 - lr: 0.0010
Epoch 2/52
52/52 [==============================] - 26s 505ms/step - loss: 0.1627 - accuracy: 0.9794 - val_loss: 0.1378 - val_accuracy: 0.9808 - lr: 0.0010
Epoch 3/52
52/52 [==============================] - 26s 493ms/step - loss: 0.1609 - accuracy: 0.9788 - val_loss: 0.1339 - val_accuracy: 0.9810 - lr: 0.0010
Epoch 4/52
52/52 [==============================] - 25s 485ms/step - loss: 0.1447 - accuracy: 0.9798 - val_loss: 0.1235 - val_accuracy: 0.9817 - lr: 0.0010
Epoch 5/52
52/52 [==============================] - 25s 484ms/step - loss: 0.1367 - accuracy: 0.9809 - val_loss: 0.1209 - val_accuracy: 0.9826 - lr: 0.0010
Epoch 6/52
52/52 [==============================] - 25s 477ms/step - loss: 0.1353 - accuracy: 0.9811 - val_loss: 0.1190 - val_accuracy: 0.9829 - lr: 0.0010
Epoch 7/52
52/52 [==============================] - 24s 462ms/step - loss: 

Forbidden: 403 POST https://storage.googleapis.com/upload/storage/v1/b/nome-do-seu-bucket/o?uploadType=resumable: {
  "error": {
    "code": 403,
    "message": "The billing account for the owning project is disabled in state closed",
    "errors": [
      {
        "message": "The billing account for the owning project is disabled in state closed",
        "domain": "global",
        "reason": "accountDisabled",
        "locationType": "header",
        "location": "Authorization"
      }
    ]
  }
}
: ('Request failed with status code', 403, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.CREATED: 201>)

In [ ]:
print('starttoken' in tokenizer.word_index)

True


In [ ]:
test_input = "Qual é a capital da França?"
test_input_seq = tokenizer.texts_to_sequences([test_input])
test_input_seq = tf.keras.preprocessing.sequence.pad_sequences(test_input_seq, maxlen=max_len, padding='post')

response = decode_sequence(test_input_seq)
print(response)

1/1 [==============================] - 0s 22ms/step
 não que que que o gente que o gente que o gente que o gente que o gente que o gente que a gente que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que o gente que o gente que o gente que o gente que o gente que o gente que o gente que o gente que o gente que o gente que o gente que o gente que o gente que


In [ ]:
print("Test question:", test_input)
print("Input sequence:", test_input_seq)

Test question: Qual é a capital da França?
Input sequence: [[  74   12    6 3283   15    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     

In [ ]:
import tensorflow as tf
import numpy as np
# Importe outras bibliotecas que você usou para pré-processar as perguntas (ex: nltk, re)

# Carregue o modelo
modelo = tf.keras.models.load_model('/content/drive/MyDrive/AI/whatsapp_response_model.h5')

def responder_pergunta(pergunta):
    # ... (rest of the function)
    pergunta_processada = pre_processar_pergunta(pergunta)
    entrada_modelo = preparar_entrada_modelo(pergunta_processada) # Call function to get entrada_modelo

    # Gere a resposta usando o modelo
    # Prepare decoder input (start token)
    decoder_input = np.zeros((1, 1))
    decoder_input[0, 0] = tokenizer.word_index['starttoken']

    # Predict using both encoder and decoder inputs
    resposta = modelo.predict([entrada_modelo, decoder_input])


    # Pós-processe a resposta, se necessário
    # Exemplo: converter de volta para texto, formatar a resposta, etc.
    resposta_final = pos_processar_resposta(resposta)

    return resposta_final

# Funções para pré-processar a pergunta e preparar a entrada do modelo
def pre_processar_pergunta(pergunta):
    # Implemente a lógica de pré-processamento aqui
    # ...
    pergunta_processada = pergunta.lower() # Exemplo: converter para minúsculas
    return pergunta_processada # Retorna a pergunta processada

def preparar_entrada_modelo(pergunta_processada):
    # Implemente a lógica de preparação da entrada aqui
    # ...
    # Crie a variável entrada_modelo aqui e preencha com os dados corretos
    entrada_modelo =  tokenizer.texts_to_sequences([pergunta_processada]) # Exemplo
    entrada_modelo = tf.keras.preprocessing.sequence.pad_sequences(entrada_modelo, maxlen=max_len, padding='post') # Exemplo
    return entrada_modelo # Retorna a entrada do modelo

# Função para pós-processar a resposta
def pos_processar_resposta(resposta, temperatura=0.2):
    resposta_decodificada = []
    for timestep in resposta:
        # Aplica a amostragem de temperatura
        token_probs = timestep[0] / temperatura
        token_probs = np.exp(token_probs) / np.sum(np.exp(token_probs))
        sampled_token_index = np.random.choice(len(token_probs), p=token_probs)

        # Verifique se sampled_token_index está dentro dos limites do vocabulário
        if sampled_token_index < len(tokenizer.word_index):
            sampled_char = tokenizer.index_word[sampled_token_index]
            resposta_decodificada.append(sampled_char)
        else:
            print("Índice de token amostrado fora dos limites:", sampled_token_index)

    # Remove tokens especiais e junta a resposta em uma string
    resposta_final = ' '.join([char for char in resposta_decodificada if char not in ['starttoken', 'endtoken']])
    return resposta_final  # Certifique-se de retornar a resposta final

# Teste o modeloé a
pergunta = "Qual a caapital da França "
resposta = responder_pergunta(pergunta)
print(resposta)

1/1 [==============================] - 1s 659ms/step
musica
